<a href="https://colab.research.google.com/github/dhkdsns20/AI-Basic/blob/main/C2_W3_Lab_1_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-1-public/blob/master/C2/W3/ungraded_lab/C2_W3_Lab_1_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Transfer Learning

In this lab, you will see how you can use a pre-trained model to achieve good results even with a small training dataset. This is called _transfer learning_ and you do this by leveraging the trained layers of an existing model and adding your own layers to fit your application. For example, you can:

1. just get the convolution layers of one model
2. attach some dense layers onto it
3. train just the dense network
4. evaluate the results

Doing this will allow you to save time building your application because you will essentially skip weeks of training time of very deep networks. You will just use the features it has learned and tweak it for your dataset. Let's see how these are done in the next sections.

**IMPORTANT NOTE:** This notebook is designed to run as a Colab. Running the notebook on your local machine might result in some of the code blocks throwing errors.

 요약: 전이 학습 실습 (Ungraded Lab: Transfer Learning)
이 실습에서는 전이 학습(Transfer Learning) 을 통해 소량의 데이터로도 좋은 성능을 내는 방법을 배웁니다. 전이 학습은 기존의 사전 학습된 모델(예: InceptionV3) 을 활용하여 학습 시간을 단축하고 성능을 향상시키는 기법입니다.



## Setup the pretrained model

You will need to prepare pretrained model and configure the layers that you need. For this exercise, you will use the convolution layers of the [InceptionV3](https://arxiv.org/abs/1512.00567) architecture as your base model. To do that, you need to:

1. Set the input shape to fit your application. In this case. set it to `150x150x3` as you've been doing in the last few labs.

2. Pick and freeze the convolution layers to take advantage of the features it has learned already.

3. Add dense layers which you will train.

Let's see how to do these in the next cells.

First, in preparing the input to the model, you want to fetch the pretrained weights of the `InceptionV3` model and remove the fully connected layer at the end because you will be replacing it later. You will also specify the input shape that your model will accept. Lastly, you want to freeze the weights of these layers because they have been trained already.

요약: 사전 학습된 모델 설정 (InceptionV3 기반 전이 학습 준비)
이 단계에서는 사전 학습된 InceptionV3 모델의 합성곱 층만 가져와서 새로운 작업에 맞게 모델을 구성합니다.

수행할 작업
1. 입력 크기 설정

  (150, 150, 3) : RGB 이미지로 고정 (이전 실습과 동일)

2. 사전 학습된 InceptionV3 모델 불러오기

  include_top=False → 마지막 Fully Connected(FC) 층 제거

  weights='imagenet' → ImageNet으로 학습된 가중치 사용

3. 합성곱 층 고정 (Freeze)

  기존에 학습된 합성곱 계층들은 가중치를 업데이트하지 않도록 고정

4. 새로운 Dense 층 추가

  이후 단계에서 학습할 새로운 출력층을 추가

In [ ]:
# Download the pre-trained weights. No top means it excludes the fully connected layer it uses for classification.
# 사전 학습된 InceptionV3 모델의 합성곱 층만 가져와서 새로운 작업에 맞게 모델을 구성합니다.
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

In [ ]:
!pip install tensorflow

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers

# 사전 학습된 가중치 파일 경로
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

# InceptionV3 모델 불러오기 (FC 층 제외, 가중치는 나중에 로드)
pre_trained_model = InceptionV3(input_shape=(150, 150, 3),
                                 include_top=False,
                                 weights=None)

# 사전 학습된 가중치 로드
pre_trained_model.load_weights(local_weights_file)

# 모든 레이어의 학습을 비활성화 (고정)
for layer in pre_trained_model.layers:
    layer.trainable = False


You can see the summary of the model below. You can see that it is a very deep network. You can then select up to which point of the network you want to use. As Laurence showed in the exercise, you will use up to `mixed7` as your base model and add to that. This is because the original last layer might be too specialized in what it has learned so it might not translate well into your application. `mixed7` on the other hand will be more generalized and you can start with that for your application. After the exercise, feel free to modify and use other layers to see what the results you get.

사전 학습된 InceptionV3 모델의 구조를 요약(summary) 해보고, 그 중에서 어디까지 사용할지를 선택합니다. 여기서는 mixed7 레이어까지를 기반 모델의 출력으로 사용합니다.

주요 포인트
- InceptionV3는 매우 깊은(deep) 신경망으로 구성되어 있음

- 마지막 출력층은 ImageNet 데이터에 특화된 특성을 학습했기 때문에, 다른 문제에 과적합(overfitting) 될 수 있음

- mixed7 레이어는 비교적 일반적인 특징(generalized features) 을 추출하는 위치이기 때문에, 전이 학습의 출발점으로 적절함

참고 팁
- mixed7 이전까지의 층은 이미지의 일반적인 패턴(모서리, 질감 등)을 학습

- 그 이후의 층은 ImageNet의 1000개 클래스에 특화된 고차원 패턴을 학습함

- 따라서 중간 지점인 mixed7까지만 사용하고, 그 이후는 새로운 문제에 맞는 층으로 직접 구성하는 것이 일반적

In [ ]:
pre_trained_model.summary()

In [ ]:
# InceptionV3 모델에서 'mixed7' 레이어를 선택
last_layer = pre_trained_model.get_layer('mixed7')

# 선택한 레이어의 출력 텐서 형태 확인
print('last layer output shape: ', last_layer.output.shape)

# 해당 레이어의 출력을 기반으로 새로운 모델 구성 예정
last_output = last_layer.output


## Add dense layers for your classifier

Next, you will add dense layers to your model. These will be the layers that you will train and is tasked with recognizing cats and dogs. You will add a [Dropout](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) layer as well to regularize the output and avoid overfitting.

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import Model

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)
# Add a final sigmoid layer for classification
x = layers.Dense  (1, activation='sigmoid')(x)

# Append the dense network to the base model
model = Model(pre_trained_model.input, x)

# Print the model summary. See your dense network connected at the end.
model.summary()

In [ ]:
# Set the training parameters
model.compile(optimizer = RMSprop(learning_rate=0.0001),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

## Prepare the dataset

Now you will prepare the dataset. This is basically the same code as the one you used in the data augmentation lab.

In [ ]:
# Download the dataset
!wget https://storage.googleapis.com/tensorflow-1-public/course2/cats_and_dogs_filtered.zip

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Model

# 1. Flatten 층: 7x7x768 → 1차원 벡터로 변환
x = layers.Flatten()(last_output)

# 2. Dense 층: 은닉층 (ReLU 활성화)
x = layers.Dense(1024, activation='relu')(x)

# 3. 출력층: sigmoid 활성화 → 이진 분류 (1개의 출력값)
x = layers.Dense(1, activation='sigmoid')(x)

# 4. 모델 구성: 입력은 pre_trained_model의 입력, 출력은 새로 만든 x
model = Model(pre_trained_model.input, x)

# 5. 모델 컴파일
model.compile(optimizer=RMSprop(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['acc'])  # 정확도 추적


In [ ]:
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 📦 1. 데이터 압축 파일(zip) 열기 및 해제
zip_ref = zipfile.ZipFile("./cats_and_dogs_filtered.zip", 'r')  # zip 파일 열기
zip_ref.extractall("tmp/")  # 'tmp/' 폴더에 압축 해제
zip_ref.close()  # 파일 닫기

# 📁 2. 데이터 경로 설정
base_dir = 'tmp/cats_and_dogs_filtered'  # 기본 데이터 폴더 경로

train_dir = os.path.join(base_dir, 'train')  # 훈련 데이터 경로
validation_dir = os.path.join(base_dir, 'validation')  # 검증 데이터 경로

# 🐱 고양이/강아지 이미지 디렉토리 경로 설정 (훈련용)
train_cats_dir = os.path.join(train_dir, 'cats')  # 훈련용 고양이 이미지 폴더
train_dogs_dir = os.path.join(train_dir, 'dogs')  # 훈련용 강아지 이미지 폴더

# 🐱 고양이/강아지 이미지 디렉토리 경로 설정 (검증용)
validation_cats_dir = os.path.join(validation_dir, 'cats')  # 검증용 고양이 이미지 폴더
validation_dogs_dir = os.path.join(validation_dir, 'dogs')  # 검증용 강아지 이미지 폴더

# 🔁 3. 훈련 데이터용 ImageDataGenerator 정의 (데이터 증강 포함)
train_datagen = ImageDataGenerator(
    rescale=1./255.,              # 픽셀값을 0~1 사이로 정규화
    rotation_range=40,           # 무작위 회전
    width_shift_range=0.2,       # 수평 이동
    height_shift_range=0.2,      # 수직 이동
    shear_range=0.2,             # 기울이기 변환
    zoom_range=0.2,              # 확대/축소
    horizontal_flip=True         # 수평 뒤집기
)

# ❗ 검증 데이터는 증강 없이 정규화만 수행
test_datagen = ImageDataGenerator(rescale=1.0/255.)

# 🚀 4. 훈련용 이미지 불러오기
train_generator = train_datagen.flow_from_directory(
    train_dir,                   # 훈련 데이터 폴더 경로
    batch_size=20,              # 배치 크기
    class_mode='binary',        # 이진 분류이므로 binary 모드
    target_size=(150, 150)      # 이미지 크기 조정
)

# 🚀 5. 검증용 이미지 불러오기
validation_generator = test_datagen.flow_from_directory(
    validation_dir,             # 검증 데이터 폴더 경로
    batch_size=20,              # 배치 크기
    class_mode='binary',        # 이진 분류
    target_size=(150, 150)      # 이미지 크기 조정
)


## Train the model

With that, you can now train the model. You will do 20 epochs and plot the results afterwards.

In [ ]:
# 🧪 모델 학습 시작
history = model.fit(
    train_generator,             # 훈련용 이미지 제너레이터
    validation_data=validation_generator,  # 검증용 이미지 제너레이터
    steps_per_epoch=100,        # 한 에폭당 훈련 배치 수 (예: 총 2000장 이미지 / 배치 20 = 100 스텝)
    epochs=20,                   # 총 학습 반복 횟수
    validation_steps=50,        # 한 에폭당 검증 배치 수 (예: 총 1000장 이미지 / 배치 20 = 50 스텝)
    verbose=2                    # 학습 중 출력 모드 (2는 에폭 단위로 간결하게 출력)
)


## Evaluate the results

You will use the same code to plot the results. As you can see, the validation accuracy is also trending upwards as your training accuracy improves. This is a good sign that your model is no longer overfitting!

In [ ]:
import matplotlib.pyplot as plt

# 학습 이력(history)에서 정확도와 손실 값 추출
acc = history.history['acc']           # 훈련 정확도
val_acc = history.history['val_acc']   # 검증 정확도
loss = history.history['loss']         # 훈련 손실
val_loss = history.history['val_loss'] # 검증 손실

# 에폭 수에 대한 인덱스 생성 (예: range(20) → 0~19)
epochs = range(len(acc))

# 🎯 정확도 시각화
plt.plot(epochs, acc, 'r', label='Training accuracy')        # 훈련 정확도 (빨간색)
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')  # 검증 정확도 (파란색)
plt.title('Training and Validation Accuracy')  # 그래프 제목
plt.legend(loc=0)  # 범례 위치 자동 설정
plt.figure()  # 새 그래프 생성

# 🎯 손실 시각화
plt.plot(epochs, loss, 'r', label='Training loss')        # 훈련 손실
plt.plot(epochs, val_loss, 'b', label='Validation loss')  # 검증 손실
plt.title('Training and Validation Loss')  # 그래프 제목
plt.legend(loc=0)  # 범례 위치 자동 설정

# 📈 그래프 출력
plt.show()
